Attempt to make a simplier vae for the medmnist dataset

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import torchvision
from torchvision import transforms
from torch.utils.data import DataLoader, random_split
from torch import nn
import torch.nn.functional as F
import torch.optim as optim

import medmnist
from medmnist.dataset import PneumoniaMNIST  # Dataset
from medmnist.dataset import OrganAMNIST, OrganCMNIST, OrganSMNIST
from medmnist import INFO, Evaluator

from torchvision.utils import save_image

In [ ]:
# data_flag = 'pneumoniamnist'
data_flag = 'organamnist'
# data_flag = 'organcmnist'
# data_flag = 'organsmnist'
download = True

num_epochs = 50
BATCH_SIZE = 128
lr = 0.005

info = INFO[data_flag]
task = info['task']
n_channels = info['n_channels']
n_classes = len(info['label'])

DataClass = getattr(medmnist, info['python_class'])


In [ ]:
# Do some preprocessing on the data
data_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[.5], std=[.5])
])

# Split the Data
train_dataset = DataClass(
    split='train', transform=data_transform, download=download)
valid_dataset = DataClass(
    split='val',   transform=data_transform, download=download)
test_dataset = DataClass(
    split='test',  transform=data_transform, download=download)

pil_dataset = DataClass(split='train', download=download)

# Convert to DataLoader
train_loader = DataLoader(dataset=train_dataset,
                          batch_size=BATCH_SIZE,   shuffle=True)
valid_loader = DataLoader(dataset=valid_dataset,
                          batch_size=2*BATCH_SIZE, shuffle=True)
train_loader_at_eval = DataLoader(
    dataset=train_dataset, batch_size=2*BATCH_SIZE, shuffle=False)
test_loader = DataLoader(dataset=test_dataset,
                         batch_size=2*BATCH_SIZE, shuffle=False)


In [ ]:
# print(train_dataset)
# print(train_loader.dataset[0])

In [ ]:
class VAE(nn.Module):
    def __init__(self, x_dim, h_dim1, h_dim2, z_dim):
        super(VAE, self).__init__()
        
        # Encoder
        self.fc1 = nn.Linear(x_dim, h_dim1)
        self.fc2 = nn.Linear(h_dim1, h_dim2)
        self.fc31 = nn.Linear(h_dim2, z_dim)
        self.fc32 = nn.Linear(h_dim2, z_dim)
        # Decoder
        self.fc4 = nn.Linear(z_dim, h_dim2)
        self.fc5 = nn.Linear(h_dim2, h_dim1)
        self.fc6 = nn.Linear(h_dim1, x_dim)
        
    def encoder(self, x):
        h = F.relu(self.fc1(x))
        h = F.relu(self.fc2(h))
        return self.fc31(h), self.fc32(h) # mu, log_var
    
    def sampling(self, mu, log_var):
        std = torch.exp(0.5*log_var)
        eps = torch.randn_like(std)
        return eps.mul(std).add_(mu) # return z sample
        
    def decoder(self, z):
        h = F.relu(self.fc4(z))
        h = F.relu(self.fc5(h))
        return F.sigmoid(self.fc6(h)) 
    
    def forward(self, x):
        mu, log_var = self.encoder(x.view(-1, 784))
        z = self.sampling(mu, log_var)
        return self.decoder(z), mu, log_var

# build model
vae = VAE(x_dim=784, h_dim1= 512, h_dim2=256, z_dim=2)
if torch.cuda.is_available():
    vae.cuda()

In [ ]:
optimizer = optim.Adam(vae.parameters())
# return reconstruction error + KL divergence losses
def loss_function(recon_x, x, mu, log_var):
    # BCE = F.binary_cross_entropy(recon_x, x.view(-1, 784), reduction='sum')
    BCE = F.cross_entropy(recon_x, x.view(-1, 784), reduction='sum')
    KLD = -0.5 * torch.sum(1 + log_var - mu.pow(2) - log_var.exp())
    return (BCE + KLD) / x.size(0)


In [ ]:
def train(epoch):
    vae.train()
    train_loss = 0
    for batch_idx, (data, _) in enumerate(train_loader):
        data = data.cuda()
        optimizer.zero_grad()

        recon_batch, mu, log_var = vae(data)
        loss = loss_function(recon_batch, data, mu, log_var)

        loss.backward()
        train_loss += loss.item()
        optimizer.step()

        if batch_idx % 50 == 0:
            print(
                f'Train Epoch: {epoch+1} [{batch_idx * len(data)}/{len(train_loader.dataset)} ({100. * batch_idx / len(train_loader):.0f}%)]\tLoss: {(loss.item() / len(data)):.6f}')
    print(f'====> Epoch: {epoch+1} Average loss: {(train_loss / len(train_loader.dataset)):.4f}')


In [ ]:
def test():
    vae.eval()
    test_loss = 0
    with torch.no_grad():
        for data, _ in valid_loader:
            data = data.cuda()
            recon, mu, log_var = vae(data)

            # sum up batch loss
            test_loss += loss_function(recon, data, mu, log_var).item()

    test_loss /= len(valid_loader.dataset)
    print('====> Test set loss: {:.4f}'.format(test_loss))


In [ ]:
print(len(valid_loader.dataset))
print(len(train_loader.dataset))

In [ ]:
device = torch.device(
    "cuda") if torch.cuda.is_available() else torch.device("cpu")

In [22]:
for epoch in range(num_epochs):
    train(epoch)
    test()

Train Epoch: 4 [19200/34581 (55%)]	Loss: -1.687656
Train Epoch: 4 [25600/34581 (74%)]	Loss: -3.791177


In [1]:
with torch.no_grad():
    z = torch.randn(64, 2).cuda()
    sample = vae.decoder(z).cuda()
    
    save_image(sample.view(64, 1, 28, 28), './samples/sample_' + '.png')

NameError: name 'torch' is not defined